Código com algumas alterações de: https://amandeepsaluja.com/extracting-job-information-from-linkedin-jobs-using-beautifulsoup-and-selenium/

In [1]:
# importing packages
import pandas as pd
import re
import numpy as np

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [2]:
# replace variables here.
# engenharia+mecanica NOT ("seleção+engenharia" OR "divulga+vagas")
url = "https://www.linkedin.com/jobs/search?keywords=engenharia%2Bmecanica%20NOT%20%28%22Sele%C3%A7%C3%A3o%2BEngenharia%22%20OR%20%22Divulga%2BVagas%22%29&location=Brasil&geoId=106057199&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0"
no_of_jobs = 500

In [3]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome(r'C:\Users\marcellohro\Downloads\chromedriver.exe')
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [4]:
# to show more jobs. Depends on number of jobs selected
i = 2
while i <= (no_of_jobs/25): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(5)

In [5]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 625 jobs.


In [6]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []

In [7]:
# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

In [72]:
# setting up list for job information
job_desc = []
level = []
emp_type = []
functions = []
industries = []

In [73]:
# for loop for job description and criterias
for x in range(1,601):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(2)
    
    # clicking on show more
    try:
        job_xpath = '/html/body/main/section/div[2]/section[2]/div[2]/section/button[1]/icon'
        driver.find_element_by_xpath(job_xpath).click()
    except Exception:
        pass
    try:
        job_xpath = '/html/body/main/section/div[2]/section[2]/div/section/button[1]/icon'
        driver.find_element_by_xpath(job_xpath).click()
    except Exception:
        pass
    sleep(0.5)
        
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
        
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    try:
        industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
        industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
        industries.append(industry_type)
    except Exception:
        industries.append(0)

In [1]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

In [83]:
job_data = pd.DataFrame({'Job ID': job_id[391:600],
'Date': post_date[391:600],
'Company Name': company_name[391:600],
'Post': post_title[391:600],
'Location': job_location[391:600],
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

In [ ]:
# # cleaning description column
# job_data['Description'] = job_data['Description'].str.replace('\n',' ')

In [89]:
job_data.to_csv('LinkedIn Job Data.csv', index=0)